# Association Rule: Heuristic Approachimport numpy as np

#### Loading necessary libraries

In [342]:
import pandas as pd
import numpy as np
import os
import string
import sys
from IPython.display import display
from nltk.corpus import stopwords
from collections import Counter 
from itertools import combinations, permutations, groupby

# functions to detect bi-grams
from gensim.models.phrases import Phrases, Phraser
from nltk.tokenize import word_tokenize #Tokenize words
from nltk.stem import WordNetLemmatizer #Lemmatization

In [343]:
# import nltk
# nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')

## 1. Reading data

### 1.1 Reading features for each of the paragraphs - Entities and Sentiments

In [2]:
#text_feat = pd.read_csv("text_features.csv",encoding='ISO-8859-1')

In [3]:
#text_feat.head()

### 1.2 Reading the actual content of each paragraph

In [20]:
textdatanew = pd.read_csv("textdatanew.csv",encoding='ISO-8859-1')
textdatanew.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...
2,a001,bio04,3,Cleopatra might have responded with a brillian...
3,a001,bio04,4,Caesar was then above fifty years of age. His ...
4,a001,bio04,5,For three years Cleopatra reigned with little ...


## 2. Analyzing and Processing data

Splitting the paragraph to have each word in a single record <br>
Also, using lemmatization

In [83]:
wordnet_lemmatizer = WordNetLemmatizer()

#### Word Tokenization and Lemmatization

In [89]:
### Tokenizing the words based on the word tokenizer function in nltk and removing the punctuations
textdatanew['Word_split'] = textdatanew.ParagraphText.apply(lambda x: [words for words in word_tokenize(x) if words not in string.punctuation])
textdatanew['Word_split_lemma'] = textdatanew.ParagraphText.apply(lambda x: [wordnet_lemmatizer.lemmatize(words) for words in word_tokenize(x) if words not in string.punctuation])

In [90]:
textdatanew.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText,Word_split,Word_split_lemma
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...,"[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ..."
2,a001,bio04,3,Cleopatra might have responded with a brillian...,"[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might, have, responded, with, a, b..."
3,a001,bio04,4,Caesar was then above fifty years of age. His ...,"[Caesar, was, then, above, fifty, years, of, a...","[Caesar, wa, then, above, fifty, year, of, age..."
4,a001,bio04,5,For three years Cleopatra reigned with little ...,"[For, three, years, Cleopatra, reigned, with, ...","[For, three, year, Cleopatra, reigned, with, l..."


#### Bi-gram detector

#### Bi-Gram for tokenized words

In [111]:
word_split_sent = textdatanew.Word_split.values
bigram = Phrases(word_split_sent, delimiter=b' ',min_count=10)
bigram_phraser = Phraser(bigram)

#### Bi-Gram for lemmatized words

In [112]:
word_lemma_sent = textdatanew.Word_split_lemma.values
bigram_lemma = Phrases(word_lemma_sent, delimiter=b' ',min_count=10)
bigram_phraser_lemma = Phraser(bigram_lemma)

#### Detecting bi-grams and adding a new column with bi-grams included

In [338]:
textdatanew['Word_split_new'] = textdatanew.Word_split.apply(lambda x: bigram_phraser[x])
textdatanew['Word_split_POS'] = textdatanew.Word_split_new.apply(lambda x: nltk.pos_tag(x))

In [339]:
textdatanew.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText,Word_split,Word_split_lemma,Word_split_new,Word_split_lemma_new,Unique_id,Word_split_POS
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...",a001_bio04_1,"[(A, DT), (FRENCH, JJ), (philosopher, NN), (mo..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...,"[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ...","[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ...",a001_bio04_2,"[(Cleopatra, NNP), (was, VBD), (joint, JJ), (h..."
2,a001,bio04,3,Cleopatra might have responded with a brillian...,"[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might have, responded, with, a, br...","[Cleopatra, might have, responded, with, a, br...",a001_bio04_3,"[(Cleopatra, NNP), (might have, NN), (responde..."
3,a001,bio04,4,Caesar was then above fifty years of age. His ...,"[Caesar, was, then, above, fifty, years, of, a...","[Caesar, wa, then, above, fifty, year, of, age...","[Caesar, was, then, above, fifty years, of, ag...","[Caesar, wa, then, above, fifty year, of, age,...",a001_bio04_4,"[(Caesar, NNP), (was, VBD), (then, RB), (above..."
4,a001,bio04,5,For three years Cleopatra reigned with little ...,"[For, three, years, Cleopatra, reigned, with, ...","[For, three, year, Cleopatra, reigned, with, l...","[For, three years, Cleopatra, reigned, with, l...","[For, three year, Cleopatra, reigned, with, li...",a001_bio04_5,"[(For, IN), (three years, NNS), (Cleopatra, NN..."


In [114]:
textdatanew['Word_split_lemma_new'] = textdatanew.Word_split_lemma.apply(lambda x: bigram_phraser_lemma[x])

In [110]:
textdatanew.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText,Word_split,Word_split_lemma,Word_split_new,Word_split_lemma_new
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...,"[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ...","[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ..."
2,a001,bio04,3,Cleopatra might have responded with a brillian...,"[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might have, responded, with, a, br...","[Cleopatra, might have, responded, with, a, br..."
3,a001,bio04,4,Caesar was then above fifty years of age. His ...,"[Caesar, was, then, above, fifty, years, of, a...","[Caesar, wa, then, above, fifty, year, of, age...","[Caesar, was, then, above, fifty years, of, ag...","[Caesar, wa, then, above, fifty year, of, age,..."
4,a001,bio04,5,For three years Cleopatra reigned with little ...,"[For, three, years, Cleopatra, reigned, with, ...","[For, three, year, Cleopatra, reigned, with, l...","[For three, years, Cleopatra, reigned, with, l...","[For three, year, Cleopatra, reigned, with, li..."


In [155]:
textdatanew.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText,Word_split,Word_split_lemma,Word_split_new,Word_split_lemma_new
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...,"[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ...","[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ..."
2,a001,bio04,3,Cleopatra might have responded with a brillian...,"[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might have, responded, with, a, br...","[Cleopatra, might have, responded, with, a, br..."
3,a001,bio04,4,Caesar was then above fifty years of age. His ...,"[Caesar, was, then, above, fifty, years, of, a...","[Caesar, wa, then, above, fifty, year, of, age...","[Caesar, was, then, above, fifty years, of, ag...","[Caesar, wa, then, above, fifty year, of, age,..."
4,a001,bio04,5,For three years Cleopatra reigned with little ...,"[For, three, years, Cleopatra, reigned, with, ...","[For, three, year, Cleopatra, reigned, with, l...","[For, three years, Cleopatra, reigned, with, l...","[For, three year, Cleopatra, reigned, with, li..."


#### Creating Unique Id

In [156]:
textdatanew['Unique_id'] = textdatanew.CollectionID+"_"+textdatanew.BiographyID+"_"+textdatanew.ParagraphNo.apply(str)

In [167]:
textdatanew.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText,Word_split,Word_split_lemma,Word_split_new,Word_split_lemma_new,Unique_id
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...","[A, FRENCH, philosopher, moralizing, on, the, ...",a001_bio04_1
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...,"[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ...","[Cleopatra, was, joint, heir, to, the, throne,...","[Cleopatra, wa, joint, heir, to, the, throne, ...",a001_bio04_2
2,a001,bio04,3,Cleopatra might have responded with a brillian...,"[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might, have, responded, with, a, b...","[Cleopatra, might have, responded, with, a, br...","[Cleopatra, might have, responded, with, a, br...",a001_bio04_3
3,a001,bio04,4,Caesar was then above fifty years of age. His ...,"[Caesar, was, then, above, fifty, years, of, a...","[Caesar, wa, then, above, fifty, year, of, age...","[Caesar, was, then, above, fifty years, of, ag...","[Caesar, wa, then, above, fifty year, of, age,...",a001_bio04_4
4,a001,bio04,5,For three years Cleopatra reigned with little ...,"[For, three, years, Cleopatra, reigned, with, ...","[For, three, year, Cleopatra, reigned, with, l...","[For, three years, Cleopatra, reigned, with, l...","[For, three year, Cleopatra, reigned, with, li...",a001_bio04_5


#### Stacking the text

##### Words without lemmatization

In [168]:
text_stack = textdatanew.apply(lambda x: pd.Series(x['Word_split_new']), axis=1).stack().reset_index(level=1, drop=True)
text_stack.name = 'Word_splits'

In [340]:
stack_cols = ['Unique_id']
text_data_words = textdatanew[stack_cols].join(text_stack)
#text_data_words['Word_splits'] = pd.Series(textdatanew2['Word_splits'], dtype=object)

## Character length
text_data_words['char_len'] = text_data_words.Word_splits.apply(lambda x: len(x))

In [341]:
text_data_words.head()

,Unique_id,Word_splits,char_len
0,a001_bio04_1,A,1
0,a001_bio04_1,FRENCH,6
0,a001_bio04_1,philosopher,11
0,a001_bio04_1,moralizing,10
0,a001_bio04_1,on,2


##### Lemmatized words

In [187]:
text_stack_lemma = textdatanew.apply(lambda x: pd.Series(x['Word_split_lemma_new']), axis=1).stack().reset_index(level=1, drop=True)
text_stack_lemma.name = 'Word_split_lemma'

In [188]:
stack_cols = ['Unique_id']
text_data_words_lemma = textdatanew[stack_cols].join(text_stack_lemma)
text_data_words_lemma['Word_split_lemma'] = pd.Series(text_data_words_lemma['Word_split_lemma'], dtype=object)

## Character length
text_data_words_lemma['char_len'] = text_data_words_lemma.Word_split_lemma.apply(lambda x: len(x))

In [189]:
text_data_words_lemma.head()

,Unique_id,Word_split_lemma,char_len
0,a001_bio04_1,A,1
0,a001_bio04_1,FRENCH,6
0,a001_bio04_1,philosopher,11
0,a001_bio04_1,moralizing,10
0,a001_bio04_1,on,2


Creating a Unique identifier for each of the paragraph

### Identifying and Removing Redundant words

In [190]:
### Tokenized Words
word_counts = Counter(text_data_words.Word_splits.apply(lambda x: x.lower()))
most_common_words = [each[0] for each in word_counts.most_common(100)]

### Lemmatized Words
word_counts_lemm = Counter(text_data_words_lemma.Word_split_lemma.apply(lambda x: x.lower()))
most_common_words_lemm = [each[0] for each in word_counts.most_common(100)]

In [191]:
text_data_words['Word_freq'] = text_data_words.Word_splits.apply(lambda x: word_counts[x.lower()])
text_data_words_lemma['Word_freq'] = text_data_words_lemma.Word_split_lemma.apply(lambda x: word_counts_lemm[x.lower()])

In [321]:
stop_words_list = stopwords.words('english')

### Adding the most common words
stop_words_list.extend(most_common_words)
stop_words_list.extend(most_common_words_lemm)

## Adding the least frequenct words
less_freq_words = text_data_words[text_data_words.Word_freq < 10].Word_splits.str.lower().values
less_freq_words_lemm = text_data_words_lemma[text_data_words_lemma.Word_freq < 10].Word_split_lemma.str.lower().values


stop_words_list.extend(less_freq_words)
stop_words_list.extend(less_freq_words_lemm)

#### Removing words with less than or equal to two characters and making sure they are not numeric
words_2_char = text_data_words[text_data_words.char_len < 3].Word_splits.unique().tolist()
words_2_char_l = text_data_words_lemma[text_data_words_lemma.char_len < 3].Word_split_lemma.unique().tolist()

words_2_char.extend(words_2_char_l)
words_2_char.extend(words_2_char)

word_2_char_to_remove = [each for each in words_2_char if ~each.isdigit()]

stop_words_list.extend(word_2_char_to_remove)

#### Removing the stop words from the original data frame

In [322]:
Text_final = text_data_words[~text_data_words.Word_splits.str.lower().isin(stop_words_list)]
Text_final_lemma = text_data_words_lemma[~text_data_words_lemma.Word_split_lemma.str.lower().isin(stop_words_list)]

In [323]:
# Text_para_final.loc[:,'Words_clean'] = Text_para_final.Word_splits.apply(lambda x: x.lower())
# Text_Para.loc[:,'Words_clean'] = Text_Para.Word_splits.apply(lambda x: x.lower())

Text_final['Words_clean'] = Text_final.Word_splits.apply(lambda x: x.lower()).values
Text_final_lemma['Words_clean'] = Text_final_lemma.Word_split_lemma.apply(lambda x: x.lower()).values

C:\Users\arvra\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\arvra\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Distribution of words greater than minimum and less than maximum threshold

In [325]:
#p = Text_Para.Word_freq[(Text_Para.Word_freq>20) & (Text_Para.Word_freq < 1500)].hist(bins = 50)

## 3. Association Rule Mining

In [277]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    #print(order_item)
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]   

In [278]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

### Applying Association Rule

In [326]:
Words = Text_final_lemma[['Unique_id','Words_clean']].drop_duplicates().set_index('Unique_id')['Words_clean'].rename('item_id')

In [327]:
%%time
rules = association_rules(Words, 0.01)  

Starting order_item:                 620471
Items with support >= 0.01:            9958
Remaining order_item:                620471
Remaining orders with 2+ items:       16526
Remaining order_item:                620383


KeyboardInterrupt: 

In [328]:
rules[rules.item_A == 'france']

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
1155232,france,maria-theresa,6,0.036280,435,2.630306,10,0.060467,0.013793,0.600000,0.228110
1154266,france,conti,6,0.036280,435,2.630306,10,0.060467,0.013793,0.600000,0.228110
1154237,france,condã©,7,0.042327,435,2.630306,13,0.078607,0.016092,0.538462,0.204714
1154057,france,bourbons,5,0.030233,435,2.630306,10,0.060467,0.011494,0.500000,0.190092
1154535,france,embassy,4,0.024187,435,2.630306,10,0.060467,0.009195,0.400000,0.152074
1155310,france,montmorency,3,0.018140,435,2.630306,8,0.048373,0.006897,0.375000,0.142569
1154056,france,bourbon,6,0.036280,435,2.630306,16,0.096747,0.013793,0.375000,0.142569
1154486,france,dubois,4,0.024187,435,2.630306,11,0.066513,0.009195,0.363636,0.138249
1155704,france,regency,5,0.030233,435,2.630306,15,0.090700,0.011494,0.333333,0.126728
1154350,france,de valois,4,0.024187,435,2.630306,12,0.072560,0.009195,0.333333,0.126728


In [349]:
df = pd.DataFrame({'value' : [1,2,None,5,None,5,6,None,None,10,1]})

In [364]:
reverse_fill = df.iloc[::-1].value.fillna(method = 'ffill')

In [365]:
fill = df.value.fillna(method = 'ffill')

In [375]:
pd.concat([fill,reverse_fill],axis = 1).apply(lambda x: np.mean(x), axis = 1)

0      1.0
1      2.0
2      3.5
3      5.0
4      5.0
5      5.0
6      6.0
7      8.0
8      8.0
9     10.0
10     1.0
dtype: float64